# DepthAI Object Detection Tutorial

<img src="https://docs.luxonis.com/images/depthai_logo.png" width="500">

Welcome to DepthAI! 

In this tutorial we will go through the basic training of an object detection model. The model will be trained to recognize 3 fruits: apples, bananas and oranges.

The model is a pretrained Mobilenet SSD v2 from the Tensorflow Object Detection API model zoo. The framework used for training is TensorFlow 1.15.2.
Will run through the following steps:


*   Install the libraries
*   Clone the github repo with the training data(images)
*   Train the model on the new images
*   Run inference on a few images to see what the model can detect
*   As an optional step, convert the model for OpenVINO inference

You can make a copy of this tutorial: File-> Save a copy in Drive






In [ ]:
while True:
    pass

# Install Libraries
# Press Shift+Enter to run cells
Some cells are commented out (all text is green), so they will not run the code inside.
To uncomment code inside a cell, select all the code then press Ctrl + /

In [ ]:
# %%capture
#After this cell executes runtime will restart, ignore the crash message, continue running cells starting with the one below
!pip install numpy==1.17.5;
# !pip install tensorflow-gpu==1.15.0;
import os
os.kill(os.getpid(), 9)


In [ ]:
%tensorflow_version 1.x
!pip install tf_slim

In [ ]:
# For the fruit model included in the repo below we have 240 training images
# For faster training time, images should be resized to 300x300 and then annotated
# Images should contain the objects of interest at various scales, angles, lighting conditions, locations
# For acceptable results - mAP@0.5 of 0.9 the model was trained with batch size of 24
# and 5000 steps. this takes about 1h using 2 augmentations. 
# using 5 augmentations it takes about 2h 
num_steps = 10000 #1000 # A step means using a single batch of data. larger batch, less steps required
#Number of evaluation steps.
num_eval_steps = 50
#Batch size 24 is a setting that generally works well. can be changed higher or lower 
MODELS_CONFIG = {
        'ssd_mobilenet_v2': {
        'model_name': 'ssd_mobilenet_v2_coco_2018_03_29',
        'pipeline_file': 'ssd_mobilenet_v2_coco.config',
        'batch_size': 32 #24
    },
        'ssd_resnet_50': {
        'model_name': 'ssd_resnet50_v1_fpn_shared_box_predictor_640x640_coco14_sync_2018_07_03',
        'pipeline_file': 'ssd_resnet50_v1_fpn_shared_box_predictor_640x640_coco14_sync.config',
        'batch_size': 8
    },
        'ssd_inception_v2': {
        'model_name': 'ssd_inception_v2_coco_2018_01_28',
        'pipeline_file': 'ssd_inception_v2_coco.config',
        'batch_size': 32
    }
}
#selected_model = 'ssd_mobilenet_v2'
#selected_model = 'ssd_resnet_50'
selected_model = 'ssd_inception_v2'

# Name of the object detection model to use.
MODEL = MODELS_CONFIG[selected_model]['model_name']

# Name of the pipline file in tensorflow object detection API.
pipeline_file = MODELS_CONFIG[selected_model]['pipeline_file']

# Training batch size fits in Colab's GPU memory for selected model.
batch_size = MODELS_CONFIG[selected_model]['batch_size']

## Clone the `object_detection_demo_flow` repository

In [ ]:
repo_url = 'https://github.com/GotG/object_detection_demo_flow'
import os
%cd /content
repo_dir_path = os.path.abspath(os.path.join('.', os.path.basename(repo_url)))
!git clone {repo_url}
%cd {repo_dir_path}
!git pull

In [ ]:
# #quick check for training data files
# !ls /content/object_detection_demo_flow/data/images/train

In [ ]:
from google.colab import files

%cd /content
print("Upload the allimages.zip file")
files.upload()

!unzip allimages.zip -d ./allimages

In [ ]:
%cd /content

# clear original images
!rm object_detection_demo_flow/test/*
!rm object_detection_demo_flow/data/images/train/*
!rm object_detection_demo_flow/data/images/test/*
!rm object_detection_demo_flow/data/images/final_test/*

# instead, copy my images :D
!cp allimages/* object_detection_demo_flow/data/images/train/
!cp allimages/* object_detection_demo_flow/data/images/test/
!cp allimages/* object_detection_demo_flow/data/images/final_test/

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

In [ ]:
"""
%cd /content

# clear original images
!rm object_detection_demo_flow/test/*
!rm object_detection_demo_flow/data/images/train/*
!rm object_detection_demo_flow/data/images/test/*
!rm object_detection_demo_flow/data/images/final_test/*

!mkdir allimages

import glob, os
from shutil import copyfile
os.chdir("/content/gdrive/MyDrive/TGMB/datasets/to-bee-or-not-to-bee/labels")
for file in glob.glob("*.xml"):
    #print(file)
    copyfile(f"/content/gdrive/MyDrive/TGMB/datasets/to-bee-or-not-to-bee/allimages/{file[:-4]}.jpg", f"/content/allimages/{file[:-4]}.jpg")
!cp /content/gdrive/MyDrive/TGMB/datasets/to-bee-or-not-to-bee/labels/* /content/allimages/
"""

In [ ]:
"""
%cd /content
!mkdir /content/gdrive/MyDrive/TGMB/datasets/to-bee-or-not-to-bee/1000images

# instead, copy my images :D
!cp allimages/* object_detection_demo_flow/data/images/train/
!cp allimages/* object_detection_demo_flow/data/images/test/
!cp allimages/* object_detection_demo_flow/data/images/final_test/
!cp allimages/* /content/gdrive/MyDrive/TGMB/datasets/to-bee-or-not-to-bee/1000images/
"""

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

%cd /content
!mkdir allimages
!cp /content/gdrive/MyDrive/TGMB/datasets/to-bee-or-not-to-bee/1000images/* /content/allimages/

# clear original images
!rm object_detection_demo_flow/test/*
!rm object_detection_demo_flow/data/images/train/*
!rm object_detection_demo_flow/data/images/test/*
!rm object_detection_demo_flow/data/images/final_test/*

# instead, copy my images :D
!cp allimages/* object_detection_demo_flow/data/images/train/
!cp allimages/* object_detection_demo_flow/data/images/test/
!cp allimages/* object_detection_demo_flow/data/images/final_test/

## Clone TF models which contains the Object Detection API; also install the required dependencies


In [ ]:
%%capture
%cd /content
!git clone --quiet https://github.com/tensorflow/models.git
%cd /content/models/
!git checkout 58d19c67e1d30d905dd5c6e5092348658fed80af
!apt-get install -qq protobuf-compiler python-pil python-lxml python-tk
!pip install -q Cython contextlib2 pillow lxml matplotlib
!pip install -q pycocotools
%cd /content/models/research
!protoc object_detection/protos/*.proto --python_out=.
import os
os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/research/slim/'
!python object_detection/builders/model_builder_test.py

## Prepare `tfrecord` files



In [ ]:
%%capture
%cd {repo_dir_path}

# Convert train folder annotation xml files to a single csv file,
# generate the `label_map.pbtxt` file to `data/` directory as well.
!python xml_to_csv.py -i data/images/train -o data/annotations/train_labels.csv -l data/annotations

# Convert test folder annotation xml files to a single csv.
!python xml_to_csv.py -i data/images/test -o data/annotations/test_labels.csv

# Generate `train.record`
!python generate_tfrecord.py --csv_input=data/annotations/train_labels.csv --output_path=data/annotations/train.record --img_path=data/images/train --label_map data/annotations/label_map.pbtxt

# Generate `test.record`
!python generate_tfrecord.py --csv_input=data/annotations/test_labels.csv --output_path=data/annotations/test.record --img_path=data/images/test --label_map data/annotations/label_map.pbtxt

In [ ]:
test_record_fname = '/content/object_detection_demo_flow/data/annotations/test.record'
train_record_fname = '/content/object_detection_demo_flow/data/annotations/train.record'
label_map_pbtxt_fname = '/content/object_detection_demo_flow/data/annotations/label_map.pbtxt'

## Download the (Mobilenet SSD v2) Model

In [ ]:
%cd /content/models/research

import os
import shutil
import glob
import urllib.request
import tarfile
MODEL_FILE = MODEL + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'
DEST_DIR = '/content/models/research/pretrained_model'

if not (os.path.exists(MODEL_FILE)):
    urllib.request.urlretrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)

tar = tarfile.open(MODEL_FILE)
tar.extractall()
tar.close()

os.remove(MODEL_FILE)
if (os.path.exists(DEST_DIR)):
    shutil.rmtree(DEST_DIR)
os.rename(MODEL, DEST_DIR)
!echo {DEST_DIR}
!ls -alh {DEST_DIR}

In [ ]:
#TF pretrained model checkpoint
fine_tune_checkpoint = os.path.join(DEST_DIR, "model.ckpt")
fine_tune_checkpoint

## Configuring a Training Pipeline

In [ ]:
import os
pipeline_fname = os.path.join('/content/models/research/object_detection/samples/configs/', pipeline_file)

assert os.path.isfile(pipeline_fname), '`{}` not exist'.format(pipeline_fname)
def get_num_classes(pbtxt_fname):
    from object_detection.utils import label_map_util
    label_map = label_map_util.load_labelmap(pbtxt_fname)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=90, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)
    return len(category_index.keys())

In [ ]:
if selected_model == 'ssd_resnet_50':
    print("Upload ssd_resnet50_v1_fpn_shared_box_predictor_640x640_coco14_sync.config now")
    from google.colab import files
    files.upload()
    !cp /content/models/research/ssd_resnet50_v1_fpn_shared_box_predictor_640x640_coco14_sync.config /content/models/research/object_detection/samples/configs/ssd_resnet50_v1_fpn_shared_box_predictor_640x640_coco14_sync.config


In [ ]:
import re
iou_threshold = 0.50
num_classes = get_num_classes(label_map_pbtxt_fname)
with open(pipeline_fname) as f:
    s = f.read()
with open(pipeline_fname, 'w') as f:
    
    # fine_tune_checkpoint
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)
    
    # tfrecord files train and test.
    s = re.sub(
        '(input_path: ".*?)(train.record)(.*?")', 'input_path: "{}"'.format(train_record_fname), s)
    s = re.sub(
        '(input_path: ".*?)(val.record)(.*?")', 'input_path: "{}"'.format(test_record_fname), s)

    # label_map_path
    s = re.sub(
        'label_map_path: ".*?"', 'label_map_path: "{}"'.format(label_map_pbtxt_fname), s)

    # Set training batch_size.
    s = re.sub('batch_size: [0-9]+',
               'batch_size: {}'.format(batch_size), s)

    # Set training steps, num_steps
    s = re.sub('num_steps: [0-9]+',
               'num_steps: {}'.format(num_steps), s)
    
    # Set number of classes num_classes.
    s = re.sub('num_classes: [0-9]+',
               'num_classes: {}'.format(num_classes), s)
    # Set number of classes num_classes.
    s = re.sub('iou_threshold: [0-9].[0-9]+',
               'iou_threshold: {}'.format(iou_threshold), s)
    
    f.write(s)

In [ ]:
# #Have a look at the config file with various settings
#!cat {pipeline_fname}

# Train the model

##[Optional] The cell below adds Tensorboard visualization to the training process. 
Will open in new tab.

In [ ]:
#After running this cell click on the link in the output cell to open tensorboard
#Tensoarboard will show you graphically different training parameters as the model is training
#when training finishes after the set number of steps, tensorboard can be used to see a nice summary of the training process
#Visuals will load in Tensorboard after the model has gone through a few hundred steps
model_dir = 'training/'
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
LOG_DIR = model_dir
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)
get_ipython().system_raw('./ngrok http 6006 &')
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

## Start the training

In [ ]:
# note that training is stored in /content/models/research/training/
model_dir = 'training/'

# Optionally remove content in output model directory for a fresh start.
# !rm -rf {model_dir}
# os.makedirs(model_dir, exist_ok=True)
!python -W ignore /content/models/research/object_detection/model_main.py \
    --pipeline_config_path={pipeline_fname} \
    --model_dir={model_dir} \
    --alsologtostderr \
    --num_train_steps={num_steps} \
    --num_eval_steps={num_eval_steps}

In [ ]:
#model dir check for the trained model
!ls {model_dir}

## Export a Trained Inference Graph
Once your training job is complete, you need to extract the newly trained inference graph, which will be later used to perform the object detection. This can be done as follows:

In [ ]:
#clean output_directory if necessary to start fresh:

# !rm -rf /content/object_detection_demo/fine_tuned_model/ 
# os.makedirs('/content/object_detection_demo_flow/fine_tuned_model/', exist_ok=True)

In [ ]:
%%capture
import re
import numpy as np

output_directory = './fine_tuned_model'
# output_directory = '/content/gdrive/My\ Drive/data/'

lst = os.listdir(model_dir)
lst = [l for l in lst if 'model.ckpt-' in l and '.meta' in l]
steps=np.array([int(re.findall('\d+', l)[0]) for l in lst])
last_model = lst[steps.argmax()].replace('.meta', '')

last_model_path = os.path.join(model_dir, last_model)
print(last_model_path)
!python /content/models/research/object_detection/export_inference_graph.py \
    --input_type=image_tensor \
    --pipeline_config_path={pipeline_fname} \
    --output_directory={output_directory} \
    --trained_checkpoint_prefix={last_model_path}

In [ ]:
#export directory check
# !ls {output_directory}

In [ ]:
import os
pb_fname = os.path.join(os.path.abspath(output_directory), "frozen_inference_graph.pb")
assert os.path.isfile(pb_fname), '`{}` not exist'.format(pb_fname)
# !ls -alh {pb_fname}

# Prepare the TF model for DepthAI. 
# First, we convert the model to OpenVINO 20.01 Intermediate Representation (IR)
(This can be used to run inference on OpenVINO, but for our purpose it is only a step in the preparation for DepthAI)
# In order to run the model on DepthAI modules, we then compile the IR obtained above to a .blob (via a server we set up just for that) 

## Install OpenVINO 20.01


In [ ]:
%%time
%%capture
## install tools. Open Vino takes some time to download: 10-15 min sometimes.
!sudo apt-get install -y pciutils cpio
!sudo apt autoremove
## download installation files
!wget http://registrationcenter-download.intel.com/akdlm/irc_nas/16345/l_openvino_toolkit_p_2020.1.023.tgz
path = "l_openvino_toolkit_p_2020.1.023.tgz"
# path = "/content/software/Intel OpenVINO 2019 R3.1/l_openvino_toolkit_p_2019.3.376.tgz"
## install openvino
!tar xf "{path}"
%cd l_openvino_toolkit_p_2020.1.023/
!./install_openvino_dependencies.sh && \
    sed -i 's/decline/accept/g' silent.cfg && \
    ./install.sh --silent silent.cfg

OpenVINO install check

In [ ]:
# !source /opt/intel/openvino/bin/setupvars.sh && \
#     /opt/intel/openvino/deployment_tools/demo/demo_squeezenet_download_convert_run.sh

In [ ]:
%cd /opt/intel/openvino/deployment_tools/model_optimizer/extensions/front/tf/
#openvino fixes: edit 
# Read in the file, make sure the .json corresponds to the model!!!
with open('ssd_v2_support.json', 'r') as file :
  filedata = file.read()

# Replace the target string
filedata = filedata.replace('"Postprocessor/ToFloat"', '"Postprocessor/Cast_1"')

# Write the file out again
with open('ssd_v2_support.json', 'w') as file:
  file.write(filedata)


## Convert TF model to OpenVINO IR

In [ ]:
#CONVERT TF MODEL to OPEN VINO IRv10. saved in IRv10 directory or choose name
%cd "/content/models/research/fine_tuned_model/"
!source /opt/intel/openvino/bin/setupvars.sh && \
    python /opt/intel/openvino/deployment_tools/model_optimizer/mo.py \
    --input_model frozen_inference_graph.pb \
    --tensorflow_use_custom_operations_config /opt/intel/openvino/deployment_tools/model_optimizer/extensions/front/tf/ssd_v2_support.json \
    --tensorflow_object_detection_api_pipeline_config pipeline.config \
    --reverse_input_channels \
    --output_dir ./IR_V10_fruits_mnssdv2_6k \
    --data_type FP16  
    

## We compile the IR model to a .blob for use on DepthAI modules/platform
We save the blob in the IR directory from above, corresponding to --output_dir parameter above.

The blob filename will be frozen_inference_graph.blob

In [ ]:
#No changes needed here unless using custom data.
#CHOOSE the directory where you would like to save the blob.
# I use the same --output_dir as above for the IR conversion
blob_dir = "/content/models/research/fine_tuned_model/IR_V10_fruits_mnssdv2_6k/"

#Copy the path of your .xml and .bin files. For that, you can look at the IR
#conversion output cell, select and copy from:
#[SUCCESS] XML file and bin file paths.
#Or you can choose to compile other .xml .bin files from a different location
#
xmlfile = "/content/models/research/fine_tuned_model/./IR_V10_fruits_mnssdv2_6k/frozen_inference_graph.xml"
binfile = "/content/models/research/fine_tuned_model/./IR_V10_fruits_mnssdv2_6k/frozen_inference_graph.bin"

import requests

#For openvino 20.01 use this link to compile the blobb
url = "http://69.164.214.171:8080"


#open vino 20.02 link:
# url = "69.164.214.171:8081"

#payload = {'compiler_params': '-ip U8 -VPU_MYRIAD_PLATFORM VPU_MYRIAD_2480 -VPU_NUMBER_OF_SHAVES 4 -VPU_NUMBER_OF_CMX_SLICES 4'}
payload = {'compiler_params': '-ip U8 -VPU_MYRIAD_PLATFORM VPU_MYRIAD_2480 -VPU_NUMBER_OF_SHAVES 8 -VPU_NUMBER_OF_CMX_SLICES 8'}
files = [
  ('definition', open(xmlfile,'rb')),
  ('weights', open(binfile,'rb'))
]
# headers = {
#   'Content-Type': 'application/json'
# }
response = requests.request("POST", url, data = payload, files = files)
blobnameraw = response.headers.get('Content-Disposition')
blobname = blobnameraw[blobnameraw.find('='):][1:]
with open(blob_dir + blobname, 'wb') as f:
  f.write(response.content)

##Now you can download your .blob file and run it on the DepthAI module/platform
To download locally, use the file explorer on the left to locate the file in the --output_dir folder, then right click download. Colab takes a few seconds to prepare the file, then the download prompt will appear.

## To run the .blob in DepthAI, follow the tutorial:
https://docs.luxonis.com/en/latest/pages/tutorials/hello_world/



[Optional] to convert the blob locally, download the IR files .bin and .xml and follow these instructions:

https://docs.luxonis.com/en/latest/pages/tutorials/local_convert_openvino/

In [ ]:
#compress the IR_V10 folder. it is in content/models/research/fine_tuned_model
!tar cvf IR_V10_fruits_mnssdv2_6k.tar.gz IR_V10_fruits_mnssdv2_6k


In [ ]:
#download the compressed IRv10 folder locally
#or can use file navigator on the left to move it to your gdrive
from google.colab import files
files.download("IR_V10_fruits_mnssdv2_6k")



```
# This is formatted as code
```

# Intel OpenVINO toolkit Installation and Validations 2021

In [ ]:
!cd /content

In [ ]:
print("This will take a while...")
!wget -O openvino_key https://apt.repos.intel.com/openvino/2021/GPG-PUB-KEY-INTEL-OPENVINO-2021?elq_cid=6770273_ts1607381885691&erpm_id=9830841_ts1607381885691&elq_cid=6770273_ts1607381960247&erpm_id=9830841_ts1607381960247 
!apt-key add openvino_key
#!apt-key -q list
!echo "deb https://apt.repos.intel.com/openvino/2021 all main" | tee /etc/apt/sources.list.d/intel-openvino-2021.list
!apt update -q
!apt-cache search intel-openvino-dev-ubuntu18
!apt-get install intel-openvino-dev-ubuntu18-2021.2.200 -y -q
!pip install -U --no-deps --quiet openvino

#unknown bug - this is just a patch
!cp /opt/intel/openvino_2021/deployment_tools/inference_engine/external/tbb/lib/libtbb.so /usr/lib/x86_64-linux-gnu/libtbb.so
!cp /opt/intel/openvino_2021/deployment_tools/inference_engine/external/tbb/lib/libtbb.so.2 /usr/lib/x86_64-linux-gnu/libtbb.so.2
!ldconfig
print("Installation Completed...")
#%env we can set environmental variable with this

#Run the validation
!ls /opt/intel/
#Run the Validation Demo code.
demo_cmd = "/opt/intel/openvino_2021/deployment_tools/demo/demo_squeezenet_download_convert_run.sh"
import os
import subprocess
import shutil
output = subprocess.check_output(demo_cmd, shell=True)
print (output.decode('utf-8'))     

## Convert to OpenVINO IR format

### Old option (do not use?!)

In [ ]:
%cd /opt/intel/openvino_2021/deployment_tools/model_optimizer/extensions/front/tf/
#openvino fixes: edit 
# Read in the file, make sure the .json corresponds to the model!!!
with open('ssd_v2_support.json', 'r') as file :
  filedata = file.read()

# Replace the target string
filedata = filedata.replace('"Postprocessor/ToFloat"', '"Postprocessor/Cast_1"')

# Write the file out again
with open('ssd_v2_support.json', 'w') as file:
  file.write(filedata)

In [ ]:
#CONVERT TF MODEL to OPEN VINO IRv10. saved in IRv10 directory or choose name
%cd "/content/models/research/fine_tuned_model/"
!source /opt/intel/openvino_2021/bin/setupvars.sh && \
    python /opt/intel/openvino_2021/deployment_tools/model_optimizer/mo.py \
    --input_model frozen_inference_graph.pb \
    --input_shape [1,300,300,3] \
    --tensorflow_object_detection_api_pipeline_config pipeline.config \
    --reverse_input_channels \
    --output_dir ./IR_V10_fruits_mnssdv2_6k \
    --data_type FP16

### New option

In [ ]:
%cd /opt/intel/openvino_2021/deployment_tools/model_optimizer/extensions/front/tf/
#openvino fixes: edit 
# Read in the file, make sure the .json corresponds to the model!!!
with open('ssd_v2_support.json', 'r') as file :
  filedata = file.read()

# Replace the target string
filedata = filedata.replace('"Postprocessor/ToFloat"', '"Postprocessor/Cast_1"')

# Write the file out again
with open('ssd_v2_support.json', 'w') as file:
  file.write(filedata)

In [ ]:
source_cmd= "/opt/intel/openvino_2021/bin/setupvars.sh"

%cd "/content/models/research/fine_tuned_model/"

# Setup model optimizer command ...
ir_name = "bee_detection"
ir_data_type = "FP16"
ir_out_dir = f"/content/IR_models/{ir_data_type}"
ir_input_shape = "[1,300,300,3]"
model_fname = "/content/models/research/fine_tuned_model/"

if selected_model == 'ssd_mobilenet_v2':
    !source /opt/intel/openvino_2021/bin/setupvars.sh && \
        python /opt/intel/openvino_2021/deployment_tools/model_optimizer/mo.py \
        --input_model frozen_inference_graph.pb \
        --input_shape [1,300,300,3] \
        --tensorflow_object_detection_api_pipeline_config pipeline.config \
        --reverse_input_channels \
        --output_dir {ir_out_dir} \
        --data_type FP16 \
        --input image_tensor \
        --output detection_classes,detection_scores,detection_boxes,num_detections \
        --tensorflow_use_custom_operations_config /opt/intel/openvino_2021/deployment_tools/model_optimizer/extensions/front/tf/ssd_v2_support.json

if selected_model == 'ssd_resnet_50':
    !source /opt/intel/openvino_2021/bin/setupvars.sh && \
        python /opt/intel/openvino_2021/deployment_tools/model_optimizer/mo.py \
        --input_model frozen_inference_graph.pb \
        --input_shape [1,640,640,3] \
        --tensorflow_object_detection_api_pipeline_config pipeline.config \
        --reverse_input_channels \
        --output_dir {ir_out_dir} \
        --data_type FP16 \
        --input image_tensor \
        --output detection_classes,detection_scores,detection_boxes,num_detections \
        --tensorflow_use_custom_operations_config /opt/intel/openvino_2021/deployment_tools/model_optimizer/extensions/front/tf/ssd_v2_support.json

mo_cmd = f"/opt/intel/openvino_2021/deployment_tools/model_optimizer/mo_tf.py \
      --saved_model_dir {model_fname} \
      --input_shape {ir_input_shape} \
      --data_type {ir_data_type} \
      --output_dir {ir_out_dir}  \
      --model_name {ir_name}\
      --reverse_input_channels\
      --input image_tensor\
      --tensorflow_object_detection_api_pipeline_config pipeline.config\
      --output detection_classes,detection_scores,detection_boxes,num_detections "
#print ("Running model optimizer to convert model to OpenVINO IR format ....")
#print("\n--".join(mo_cmd.split("--")))

#output = subprocess.check_output(source_cmd+" && "+mo_cmd, shell=True)
#print (output.decode('utf-8'))     

## Convert OpenVINO IR to Blob format 

In [ ]:
!cp /opt/intel/openvino_2021.2.200/deployment_tools/inference_engine/lib/intel64/libmyriadPlugin.so /usr/lib/x86_64-linux-gnu/libmyriadPlugin.so
#!source /opt/intel/openvino_2021/bin/setupvars.sh && /opt/intel/openvino_2021/deployment_tools/inference_engine/lib/intel64/myriad_compile -m {ir_out_dir}/frozen_inference_graph.xml -ip U8 -VPU_NUMBER_OF_SHAVES 4 -VPU_NUMBER_OF_CMX_SLICES 4
!source /opt/intel/openvino_2021/bin/setupvars.sh && /opt/intel/openvino_2021/deployment_tools/inference_engine/lib/intel64/myriad_compile -m {ir_out_dir}/frozen_inference_graph.xml -ip U8 -VPU_NUMBER_OF_SHAVES 10 -VPU_NUMBER_OF_CMX_SLICES 10

In [ ]:
!ls /content/IR_models/FP32/
#download the blob file to local directory.
from google.colab import files
files.download(f"{ir_out_dir}/frozen_inference_graph.blob")

#if you want everything - uncomment these two lines
#!zip -r flower.zip flower 
#files.download('flower.zip')